In [1]:
stock = 'NOK'
days_back = 100

In [2]:
import pandas as pd                              # 表格和数据处理
import matplotlib.pyplot as plt                  # 绘图
import seaborn as sns                            # 更多绘图功能
import pandas_datareader.data as webdata

from itertools import product                    # 一些有用的函数
from tqdm import tqdm_notebook

import datetime                                  # 日期函数

import warnings                                  # 勿扰模式
warnings.filterwarnings('ignore')

%matplotlib inline

In [3]:
now = datetime.date.today()
start = now - datetime.timedelta(days=days_back)
# df = data.DataReader(stock, "iex", start=start, end=end)
df = webdata.get_data_yahoo(stock, start=start, end=now)['Adj Close']

In [4]:
df

Date
2018-07-23    5.94
2018-07-24    5.90
2018-07-25    5.98
2018-07-26    5.55
2018-07-27    5.40
2018-07-30    5.50
2018-07-31    5.40
2018-08-01    5.43
2018-08-02    5.37
2018-08-03    5.36
2018-08-06    5.32
2018-08-07    5.39
2018-08-08    5.39
2018-08-09    5.36
2018-08-10    5.22
2018-08-13    5.19
2018-08-14    5.20
2018-08-15    5.09
2018-08-16    5.15
2018-08-17    5.23
2018-08-20    5.25
2018-08-21    5.36
2018-08-22    5.41
2018-08-23    5.43
2018-08-24    5.58
2018-08-27    5.71
2018-08-28    5.73
2018-08-29    5.71
2018-08-30    5.62
2018-08-31    5.55
              ... 
2018-09-18    5.62
2018-09-19    5.46
2018-09-20    5.52
2018-09-21    5.59
2018-09-24    5.55
2018-09-25    5.60
2018-09-26    5.53
2018-09-27    5.69
2018-09-28    5.58
2018-10-01    5.61
2018-10-02    5.48
2018-10-03    5.47
2018-10-04    5.48
2018-10-05    5.38
2018-10-08    5.32
2018-10-09    5.30
2018-10-10    5.16
2018-10-11    5.15
2018-10-12    5.25
2018-10-15    5.29
2018-10-16    5.45
2018-10

In [5]:
# 需要预测的5天日期
d = [None] * 5
j = 0

for i in range(0,7):
    if (now + datetime.timedelta(i)).weekday() < 5:
        d[j] = str(now + datetime.timedelta(i))
        j = j + 1

In [6]:
#时序数据上的线性模型
data = pd.DataFrame(df.copy().astype(float))
data.columns = ["y"]

df = pd.DataFrame({'y':[0,0,0,0,0]},
                   index = d)
data = data.append(df)

for i in range(6, 25):
    data["lag_{}".format(i)] = data.y.shift(i)

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

y = data.dropna().y # 删除n/a的数据
X = data.dropna().drop(['y'], axis=1)

# 除去需要预测的5天
train_index = int(len(X)) - 5
X_train = X.iloc[:train_index]
y_train = y.iloc[:train_index]

# 机器学习
lr = LinearRegression()
lr.fit(X_train, y_train)
prediction = lr.predict(X.tail(5))
precentage = prediction.mean()
print(prediction.mean())
print(prediction)
# plt.figure(figsize=(15, 7))
# plt.plot(prediction, "g", label="prediction", linewidth=2.0)


5.507579491319684
[5.45560701 5.35715861 5.42156748 5.63335292 5.67021143]


In [7]:
#储存数据到excel
from openpyxl import load_workbook
df = pd.DataFrame(prediction)
df = df.values.T.tolist()
stock_name = [str(stock)]
wb = load_workbook('/Users/BartSu/Desktop/StockRS_Result.xlsx')
ws = wb.worksheets[0]

ws.append(stock_name)

# Append 2 new Rows - Columns A - D
for row_data in df:
    # Append Row Values
    ws.append(row_data)

wb.save("/Users/BartSu/Desktop/StockRS_Result.xlsx")